# Create `SU`

> Joseph P. Vantassel, The University of Texas at Austin

Load data `.csv` data from LS-DYNA and write to `SU` format.

In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

import swprocess

## Constant Spacing

In [2]:
dt = 0.001
desired_time = np.arange(0, 1.5, dt)

# desired_xmin, desired_xmax, desired_dx = 5.05, 51.05, 2.
# desired_xmin, desired_xmax, desired_dx = 10.05, 56.05, 2.
desired_xmin, desired_xmax, desired_dx = 20.05, 66.05, 2.

source = swprocess.Source(x=0.05, y=0, z=0)

current_x, dx = 0.0, 0.05
current_desired_x = desired_xmin
sensors = []
for setnumber in range(14):
    fname = f"set{str(setnumber).zfill(2)}.csv"
    
    with open(fname, "r") as f:
        ncols = f.readlines()[1].count(",")
      
    df = pd.read_csv(fname, usecols=np.arange(0, ncols), names=["time", *[x for x in np.arange(ncols)]], skiprows=2, skipinitialspace=True)
    
    xs = np.arange(ncols-1)*dx + current_x
    current_x = float(xs[-1]) + dx
    
    for col, _x in enumerate(xs):
        
        if np.abs(_x - current_desired_x) > dx/2:
            continue
            
        interpolator = interp1d(x=df["time"], y=df[col], kind="cubic")
        sensor = swprocess.Sensor1C(amplitude=np.array(interpolator(desired_time), dtype=np.float32), dt=dt, x=_x, y=0, z=0)
        sensors.append(sensor)
        
        current_desired_x += desired_dx
        
        if current_desired_x > desired_xmax:
            break

    if current_desired_x > desired_xmax:
        break

array = swprocess.Array1D(sensors, source)

# array.to_file("46m_2m_-5m.su")
# array.to_file("46m_2m_-10m.su")
array.to_file("46m_2m_-20m.su")

## Non-constant Spacing

In [3]:
dt = 0.001
desired_time = np.arange(0, 1.5, dt)
# desired_xs = [5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 35, 40, 45, 50, 55, 60, 65]
desired_xs = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 24, 26, 28, 30, 35, 40, 45, 50, 55, 60, 65, 70]
desired_xs = [ x + 0.05 for x in desired_xs]
desired_x_index = 0

current_x, dx = 0.0, 0.05
current_desired_x = desired_xs[desired_x_index]

source = swprocess.Source(x=0.05, y=0, z=0)

sensors = []
for setnumber in range(14):
    fname = f"set{str(setnumber).zfill(2)}.csv"
    
    with open(fname, "r") as f:
        ncols = f.readlines()[1].count(",")
      
    df = pd.read_csv(fname, usecols=np.arange(0, ncols), names=["time", *[x for x in np.arange(ncols)]], skiprows=2, skipinitialspace=True)
    
    xs = np.arange(ncols-1)*dx + current_x
    current_x = float(xs[-1]) + dx
    
    for col, _x in enumerate(xs):
        
        if np.abs(_x - current_desired_x) > dx/2:
            continue
            
        interpolator = interp1d(x=df["time"], y=df[col], kind="cubic")
        sensor = swprocess.Sensor1C(amplitude=np.array(interpolator(desired_time), dtype=np.float32), dt=dt, x=_x, y=0, z=0)
        sensors.append(sensor)
        
        desired_x_index += 1
        
        if desired_x_index == len(desired_xs):
            break

        current_desired_x = desired_xs[desired_x_index]
        
array = swprocess.Array1D(sensors, source)

# array.to_file("60m_Xm_-5m.su")
array.to_file("60m_Xm_-10m.su")